In [1]:
import numpy as np
from numpy import loadtxt
import pandas as pd
import csv
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing training data as X_train and Y_train
file = open('dataset.csv')
csvreader = csv.reader(file, delimiter = ",")
data = []
for row in csvreader:
    data.append(row)

train_data = []
for i in range(0 , len(data)):
    col= []
    for j in range(0,len(data[i])):
        if j==len(data[i])-1:
            col.append(data[i][j])
        else:
            col.append(data[i][j])
    train_data.append(col)
file.close()
X_train = []
Y_train = []
for i in range(len(train_data)):
    cols = []
    for j in range(0,len(train_data[i])):
        if(j==len(train_data[i])-1):
            Y_train.append(train_data[i][j])
        else:
            cols.append(train_data[i][j])
    X_train.append(cols)   
for i in range(0,len(X_train)):
    for j in range(0,len(X_train[i])):
        X_train[i][j]=float(X_train[i][j])
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_train_df = pd.DataFrame(X_train, columns = ['Height','Weight','Diameter','Color'])
Y_train_df = pd.DataFrame(Y_train,columns = ['Material'])

In [3]:
#Taking out test dataset as X_test and Y_test
test_data = []
c = 0
p = 0
m = 0
k = 0
x = []
for i in range(0,len(train_data)):
    if(train_data[i][4]==' Ceramic' and c<6):
        x.append(i)
        c=c+1
        test_data.append(train_data[i])
    if(train_data[i][4]==' Plastic' and p<6):
        x.append(i)
        p=p+1
        test_data.append(train_data[i])
    if(train_data[i][4]==' Metal' and m<6):
        x.append(i)
        m=m+1
        test_data.append(train_data[i])
    if(p>=6 and c>=6 and m>=6):
            train_data.append(train_data[i])
test_df = pd.DataFrame(data = test_data)
X_test = test_df.drop([4],axis = 'columns')
Y_test = test_df[4]
X_test = np.array(X_test).astype(np.float)
Y_test = np.array(Y_test)

In [4]:
#Deleting the Test Data set form the training set
X_train_df= X_train_df.drop(X_train_df.index[x])
Y_train_df = Y_train_df.drop(Y_train_df.index[x])


In [5]:
X_train = np.array(X_train_df).astype(np.float)
Y_train  = np.array(Y_train_df)

In [15]:
#Intializing the functions for the softmax
def initialize_weights(X,Y):
    z = np.ones((len(X[1]),np.max(Y)+1))
    return z

def set_labels_int(Y):
    for i in range(0,len(Y)):
        if Y[i] ==" Metal":
            Y[i]=0
        if Y[i] == " Plastic":
            Y[i]=1
        if Y[i] == " Ceramic":
            Y[i]=2
    return Y

def reset_labels(Y):
    y=[]
    for i in range(0,len(Y)):
        if Y[i] == 0:
            y.append(" Metal")
        if Y[i] == 1:
            y.append(" Plastic")
        if Y[i] == 2:
            y.append(" Ceramic")
    return y

def set_one_hot(Y_train): 
    y_train=[]
    for i in range(0,len(Y_train)):
        arr = []
        for j in range(0,np.max(Y_train)+1):
            if Y_train[i] == j:
                arr.append(1)
            else:
                arr.append(0)
        y_train.append(arr)
    return y_train

def training(x,y,l_r,epochs):
    w = np.ones((len(x[1]),np.max(y)+1))
    y = set_one_hot(y)
    for i in range(0,epochs):
        z = np.dot(x,w)
        z = np.exp(z)
        v = np.sum(z,axis=1)
        sm = []
        for i in range(0,len(z)):
            cols = []
            for j in range(0,len(z[i])):
                cols.append(z[i][j]/v[i])
            sm.append(cols)
        sm = np.array(sm)
        sm = y - sm
        sm = np.dot(x.T,sm)
        w = w + l_r*sm
    return w
def predict(X_test,tw):
    re = np.dot(X_test,tw)
    re = np.exp(re)
    re = re / np.sum(re, axis=1, keepdims=True)
    return np.argmax(re,axis=1)
def accuracy(y_pred,y):
    c=0
    for i in range(0,len(y)):
        if y_pred[i]==y[i]:
            c=c+1
    c = (c/len(y_pred))*100
    return c

Y_train = set_labels_int(Y_train).astype(np.int)

# Accuracy without Bagging

In [16]:
tw = training(X_train,Y_train,0.1,10000)
y_pred = predict(X_test,tw)
z = set_labels_int(Y_test).astype(np.int)
print("Predicted_output")
print(y_pred)
print("Desired_output ")
print(z)
print("Accuracy")
a_wb = accuracy(y_pred,z)
print(a_wb)

Predicted_output
[2 0 2 2 1 1 1 1 1 1 1 1 1 1 1 1 2 2]
Desired_output 
[2 0 2 2 1 1 1 1 1 1 2 0 0 0 0 0 2 2]
Accuracy
66.66666666666666


# Implementing the Bagging Function

In [21]:
#Implementing the bagging function

def bagging_dataset(nod,X_train,Y_train):
    #nod is number of datasets we want to create
    X = []
    Y = []
    for i in range(0,nod):
        c = np.random.randint(len(X_train)-1,size = len(X_train))
        cols=[]
        ycols = []
        for j in range(0,len(c)):
            cols.append(X_train[c[j]])
            ycols.append(Y_train[c[j]])
        X.append(cols)
        Y.append(ycols)
    Y = np.array(Y)
    Y = Y.astype(np.int)
    X = np.array(X)
    return X,Y

def bagging(nod,X,Y):
    Y_bag_predict = []
    for i in range(0,nod):
        tw = training(X[i],Y[i],0.1,10000)
        y_pred = predict(X_test,tw)
        Y_bag_predict.append(y_pred)
    Y_bag_predict = np.array(Y_bag_predict)
    Y_bag_predict = Y_bag_predict.astype(np.int)
    return Y_bag_predict     

def pred_bagging(y):
    y_b_p = []
    for i in range(0,len(y[0])):
        x_b=[]
        x_b = y[:,i]
        values, counts = np.unique(x_b, return_counts=True)
        y_b_p.append(values[counts.argmax()])
    return y_b_p
    



# For 10 times

In [22]:
#We have successfully implemented the bagging the output 
#Now we will do it for 10 times
X,Y = bagging_dataset(10,X_train,Y_train)
y_bag_predict = bagging(10,X,Y)
a = pred_bagging(y_bag_predict)
z = set_labels_int(Y_test)
z = np.array(z)
print("Predicted_output")
print(a)
print("Desired_output")
print(z)
print("Accuracy")
a_10 = accuracy(a,z)
print(a_10)

Predicted_output
[2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2]
Desired_output
[2 0 2 2 1 1 1 1 1 1 2 0 0 0 0 0 2 2]
Accuracy
61.111111111111114


# For 50 times

In [23]:
#We have successfully implemented the bagging the output 
#Now we will do it for 50 times
X,Y = bagging_dataset(50,X_train,Y_train)
y_bag_predict = bagging(50,X,Y)
a = pred_bagging(y_bag_predict)
z = set_labels_int(Y_test)
z = np.array(z)
print("Predicted_output")
print(a)
print("Desired_output")
print(z)
print("Accuracy")
a_50 = accuracy(a,z)
print(a_50)

Predicted_output
[2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2]
Desired_output
[2 0 2 2 1 1 1 1 1 1 2 0 0 0 0 0 2 2]
Accuracy
61.111111111111114


# For 100 times

In [24]:
#We have successfully implemented the bagging the output 
#Now we will do it for 100 times
X,Y = bagging_dataset(100,X_train,Y_train)
y_bag_predict = bagging(100,X,Y)
a = pred_bagging(y_bag_predict)
z = set_labels_int(Y_test)
z = np.array(z)
print("Predicted_output")
print(a)
print("Desired_output")
print(z)
print("Accuracy")
a_100 = accuracy(a,z)
print(a_100)

Predicted_output
[2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 2, 2, 2]
Desired_output
[2 0 2 2 1 1 1 1 1 1 2 0 0 0 0 0 2 2]
Accuracy
72.22222222222221


# Outcome

In [27]:
print("Error Rates for 100 iterations "+str(100-a_100))
print("Error Rates for 50 iterations "+str(100-a_50))
print("Error Rates for 10 iterations "+str(100-a_10))
print("Error Rates without bagging "+str(100-a_wb))

Error Rates for 100 iterations 27.777777777777786
Error Rates for 50 iterations 38.888888888888886
Error Rates for 10 iterations 38.888888888888886
Error Rates without bagging 33.33333333333334


In [28]:
if a_100>=a_50 and a_100>=a_wb and a_100>=a_10:
    print("The highest accuracy is with 100 iterations")
    print(a_100)
elif a_50>a_100 and a_50>a_wb and a_50>a_10:
    print("The highest accuracy is with 50 iterations")
    print(a_50)
elif a_10>a_50 and a_10>a_wb and a_10>a_100:
    print("The highest accuracy is with 10 iterations")
    print(a_10)

The highest accuracy is with 100 iterations
72.22222222222221


# NOTE

Due to high values of epochs it takes a lot of time to run 

# Observation 

As we can see after bagging we have gradual increase of accuracy in prediction